# Geophysics anlaysis - LGCFR

In [2]:
from geophysics_utils import * 
import plotly.express as px 
import statsmodels.api as sm
import scipy.stats as stats
from pathlib import Path
import pandas as pd
import numpy as np
import re
import plotly.io as pio
pio.renderers.default = 'notebook'

%load_ext autoreload
%autoreload 2

## Parameters Setting

In [3]:
pd.set_option("display.max_columns", 100)
TANGENT_THRESHOLD = 5
PERPENDICULAR_THRESHOLD = 25
CHAINAGE_RANGE = 0.5
CONSISTENCY_ORDER = ['VS','S','F','St', 'VSt', 'H', 'VL','L','MD', 'D', 'VD', '5a', '5b','4a','4b','3a','3b','2a','2b','1a','1b']
HOVER_DATA = ['Geophysics_ID','Hole_ID', 'From_RL','Chainage', 'Velocity', 'Consistency', 'Geology_Orgin', 'perpendicular_offset']

Lab_summary_spreadsheet = "Lab_summary_final_20250909_111123.xlsx"
BH_interp_file = "BH_Interp_20250902.xlsx"

## Function definition

### plot_geophysics

In [4]:
def plot_geophysics(df, x='Chainage', y='To_RL', color='Velocity', title=None, height=500, range_color=[50,1000]):
    fig = px.scatter(df, x, y,  color=color, title=title, height=height, color_continuous_scale="jet",range_color=range_color)
    fig.update_layout(
        coloraxis_colorbar=dict(
            title="S-Velocity (m/s)",
            tickmode="linear",
            tick0=50,
            dtick=100
        )
    )
    return fig

## Import all geophysics data

- Import all geophysics data as dataframe from the folder
- Put all of them into the dictionary under the first key

In [5]:
Lab_summary = pd.read_excel(Lab_summary_spreadsheet)

# all csv files for geophysics data
folder = Path("./Geophysics_data")
list(folder.glob("*.csv"))[0].name

all_geophysics = {}
all_geophysics['geophysics_data'] = {}
geophysics_data = all_geophysics['geophysics_data']

for file in folder.glob("*.csv"):
    print(file)
    df = pd.read_csv(file)
    # print(df.columns)
    df.columns = ['Easting', 'Northing', 'Elevation', 'Chainage', 'Velocity']
    print(df.columns)
    print("\n")
    df = process_individual_geophysics(df, 5)
    all_geophysics['geophysics_data'][file.name.rstrip('.csv')] = df

Geophysics_data/163-1 DD.csv
Index(['Easting', 'Northing', 'Elevation', 'Chainage', 'Velocity'], dtype='object')


Geophysics_data/AU-GP163-01.csv
Index(['Easting', 'Northing', 'Elevation', 'Chainage', 'Velocity'], dtype='object')




In [6]:
all_geophysics['geophysics_data'].keys()

dict_keys(['163-1 DD', 'AU-GP163-01'])

## Demo for first geophysics

In [ ]:
first_item = list(all_geophysics['geophysics_data'].keys())[1]
geophysics = all_geophysics['geophysics_data'][first_item]
print(first_item)
geophysics

### Plot data

In [ ]:
fig = plot_geophysics(geophysics, title=first_item, y='From_RL')
fig.show(config={"responsive": False})

## Methodology

### Geophysics BH Register

- Getting borehole test summary - meta file

In [ ]:
Test_Summary = pd.read_excel("Test Summary.xlsx")
Test_Summary = Test_Summary.rename(columns={'Easting (m)':'Easting', 'Northing (m)':'Northing'})
try:
    Test_Summary = Test_Summary[Test_Summary['Report']!='GHD-Mott MacDonald GIR']
except:
    print("report name of <GHD-Mott MacDonald GIR> does not exist!")
Test_Summary.Report.value_counts()

In [ ]:
BH_dict = Test_Summary.set_index('Hole_ID').to_dict(orient='index')
BH_dict['TP-38']

In [ ]:
# i = 0
all_geophysics['geophysics_BH_register'] = []
geophysics_BH_register = all_geophysics['geophysics_BH_register']


for individual_geophysics_ID, individual_geophysics_dataframe in all_geophysics['geophysics_data'].items():
    length_of_geophysics_line = individual_geophysics_dataframe.Chainage.max()
    print(f"TANGENT_THRESHOLD = {TANGENT_THRESHOLD}\nPERPENDICULAR_THRESHOLD={PERPENDICULAR_THRESHOLD}")
    print(individual_geophysics_ID)
    print()
    for key,value in BH_dict.items():
        # i+=1
        BH_Easting = value['Easting']
        BH_Northing = value['Northing']
        BH_coordinates = np.array([BH_Easting, BH_Northing])
        
        tangential_distance, perpendicular_offset = offset_bh_geophysics_line(individual_geophysics_dataframe, BH_coordinates)

        if (0 < tangential_distance < length_of_geophysics_line) & (perpendicular_offset < PERPENDICULAR_THRESHOLD):
            add_to_register(geophysics_BH_register, individual_geophysics_ID, key, tangential_distance, perpendicular_offset)
            print(f"BH: {key} ({BH_Easting:.1f}, {BH_Northing:.1f}) is within {individual_geophysics_ID} at chainage {tangential_distance:.2f} with perpendicular_offset of {perpendicular_offset:.2f}")
        
        elif (tangential_distance < 0) & (abs(tangential_distance) < TANGENT_THRESHOLD)  & (perpendicular_offset < PERPENDICULAR_THRESHOLD):
            add_to_register(geophysics_BH_register, individual_geophysics_ID, key, tangential_distance, perpendicular_offset)
            print(f"BH: {key} ({BH_Easting:.1f}, {BH_Northing:.1f}) is before {individual_geophysics_ID} with offset of ({tangential_distance:.2f}, {perpendicular_offset:.2f})")

        elif (tangential_distance>length_of_geophysics_line) & ((tangential_distance-length_of_geophysics_line) < TANGENT_THRESHOLD) & (perpendicular_offset < PERPENDICULAR_THRESHOLD):
            add_to_register(geophysics_BH_register, individual_geophysics_ID, key, tangential_distance, perpendicular_offset)
            print(f"BH: {key} ({BH_Easting:.1f}, {BH_Northing:.1f}) is beyond {individual_geophysics_ID} with offset of ({tangential_distance-length_of_geophysics_line:.2f}, {perpendicular_offset:.2f})")
                
print("Geophysics_BH_registeration completed...")        

In [ ]:
geophysics_BH_register = pd.DataFrame(geophysics_BH_register)
geophysics_BH_register

### BH Interp File

In [ ]:
BH_interp = pd.read_excel(BH_interp_file)
BH_interp = BH_interp.merge(right=Test_Summary[['Hole_ID', 'Easting', 'Northing',]], how='left', left_on='Hole_ID', right_on='Hole_ID')
BH_interp['From_RL'] = BH_interp['Surface RL (m AHD)'] - BH_interp['From_mbgl']
BH_interp['To_RL'] = BH_interp['Surface RL (m AHD)'] - BH_interp['To_mbgl']
BH_interp = BH_interp.drop(columns=['From (m AHD)', 'ActivUs Interpretation', 'Chainage'])
BH_interp = BH_interp[BH_interp['Report']!='GMM 20CIGE20']
BH_interp.head(3)

### Lab Summary spreadsheet

In [ ]:
Lab_summary.head(3)

In [ ]:
RL = BH_interp[['Hole_ID', 'Surface RL (m AHD)']].drop_duplicates()

condition_UCS = Lab_summary['UCS?']=="Y"
condition_SPT = (Lab_summary['SPT?']=="Y") & (Lab_summary['Test type']=='SPT')
condition_Atterberg = Lab_summary['Atterberg?']=="Y"
Lab_summary['SPT N Value']
UCS_SPT = Lab_summary.loc[condition_UCS | condition_SPT | condition_Atterberg][['Hole_ID','From_mbgl', 'To_mbgl', 'UCS (MPa)', 'SPT N Value', 'LL (%)']]
UCS_SPT = UCS_SPT.merge(right=RL, how='left')
UCS_SPT['From_RL'] = UCS_SPT['Surface RL (m AHD)'] - UCS_SPT.From_mbgl
UCS_SPT['To_RL'] = UCS_SPT['Surface RL (m AHD)'] - UCS_SPT.To_mbgl
UCS_SPT

### Merge BH and Geophysics data

In [ ]:
geophysics_BH_register.head(3)

In [ ]:
i = 0
CHAINAGE_RANGE=CHAINAGE_RANGE
geophysics_bh_results = []
for index, row in geophysics_BH_register.iterrows(): 
    i += 1
    geophysics_ID = row.Geophysics_ID
    Hole_ID = row.Hole_ID
    # print(Hole_ID)
    geophysics_chainage = row.geophysics_chainage
    # print(f"geophysics_ID: {geophysics_ID}\nHole_ID: {Hole_ID}\ngeophysics_chainage:{geophysics_chainage}")

    geophysics_df = geophysics_data[geophysics_ID]
    
    if geophysics_chainage <=0:
        geophysics_df = geophysics_df[geophysics_df['Chainage'].between(0, 2*CHAINAGE_RANGE)]
        temp_offset = geophysics_chainage - geophysics_df.Chainage.min()
        geophysics_df.loc[:, "Chainage"] = geophysics_df.Chainage+temp_offset

    elif 0 < geophysics_chainage < geophysics_df.Chainage.max():
        geophysics_df = geophysics_df[geophysics_df['Chainage'].between(geophysics_chainage-CHAINAGE_RANGE, geophysics_chainage+CHAINAGE_RANGE)]
        
    elif geophysics_chainage>=0:
        geophysics_df = geophysics_df[geophysics_df['Chainage'].between(geophysics_df.Chainage.max()-2*CHAINAGE_RANGE, geophysics_df.Chainage.max())]
        temp_offset = geophysics_chainage - geophysics_df.Chainage.max()
        geophysics_df.loc[:, "Chainage"] = geophysics_df.Chainage+temp_offset
    
    BH_interp_df = BH_interp.groupby(by='Hole_ID').get_group(Hole_ID)

    merge_geophysics_bh_consistency(geophysics_bh_results, geophysics_ID, geophysics_df,Hole_ID, BH_interp_df)
    
    # if i >= 2:
    #     break
        
geophysics_bh_results_df = pd.DataFrame(geophysics_bh_results)
geophysics_bh_results_df.insert(loc=len(geophysics_bh_results_df.columns)-2, column='soil_type', value="")
geophysics_bh_results_df['soil_type'] = geophysics_bh_results_df['Consistency'].apply(classify_soil)
geophysics_bh_results_df = geophysics_bh_results_df.merge(right=geophysics_BH_register, how='left', on=['Geophysics_ID', 'Hole_ID'])

In [ ]:
geophysics_bh_results_df.head()

#### Demo result: AU-GP163-01

In [ ]:
Geophysics_ID = '163-1 DD'
xmin = min(geophysics_data[Geophysics_ID].Chainage.min()-20, geophysics_bh_results_df[geophysics_bh_results_df['Geophysics_ID']==Geophysics_ID].Chainage.min()-20)
xmax = max(geophysics_data[Geophysics_ID].Chainage.max()+20, geophysics_bh_results_df[geophysics_bh_results_df['Geophysics_ID']==Geophysics_ID].Chainage.max()+20)
ymin = min(geophysics_data[Geophysics_ID].To_RL.min()-1, geophysics_bh_results_df[geophysics_bh_results_df['Geophysics_ID']==Geophysics_ID].To_RL.min()-1)
ymax = max(geophysics_data[Geophysics_ID].From_RL.max()+5, geophysics_bh_results_df[geophysics_bh_results_df['Geophysics_ID']==Geophysics_ID].From_RL.max()+5)
plot_df = geophysics_bh_results_df[geophysics_bh_results_df["Geophysics_ID"]==Geophysics_ID]
Consistency_order = [c for c in CONSISTENCY_ORDER if c in plot_df["Consistency"].unique()]

In [ ]:
fig = px.scatter(plot_df, x='Chainage', y='From_RL', color='Consistency', range_x=[xmin,xmax], range_y=[ymin, ymax], category_orders={'Consistency':Consistency_order})
add_label(fig, plot_df)
fig.show(config={"responsive": False})

In [ ]:
fig = px.scatter(plot_df, x='Chainage', y='From_RL', color='Velocity', range_x=[xmin,xmax], range_y=[ymin, ymax], color_continuous_scale="jet", range_color=[50,950],)
fig.update_layout(coloraxis_colorbar=dict(title="S-Velocity (m/s)", tick0=50, dtick=100))
add_label(fig, geophysics_bh_results_df)
fig.show(config={"responsive": False})

### Merge UCS / SPT into geophysics_bh_results

In [ ]:
UCS_SPT.head(3)

In [ ]:
geophysics_bh_results_df.head(3)

In [ ]:
geophysics_bh_lab = merge_lab_into_results(geophysics_bh_results, UCS_SPT)
geophysics_bh_lab = geophysics_bh_lab.merge(right=geophysics_BH_register, how='left', on=['Geophysics_ID', 'Hole_ID'])
geophysics_bh_lab.head(3)

In [ ]:
geophysics_bh_lab.insert(loc=7, column='soil_type', value="")
geophysics_bh_lab['soil_type'] = geophysics_bh_lab['Consistency'].apply(classify_soil)
geophysics_bh_lab.head(3)

#### Demo result: AU-GP163-01

In [ ]:
plot_df_lab = geophysics_bh_lab.copy()
plot_df_lab['LL (%)'] = plot_df_lab['LL (%)'].astype('str')
plot_df_lab.head()

In [ ]:
fig = px.scatter(plot_df_lab, x='Chainage', y='From_RL', color='UCS_MPa', 
           # color_discrete_sequence=px.colors.qualitative.Set1,
           color_discrete_map={'nan':'rgba(0,0,0,0.03)'},
           range_x = [xmin, xmax],
           range_y=[ymin, ymax])
add_label(fig, plot_df_lab)
fig.show(config={"responsive": False})

## Results - Data

### All geophysics data

In [ ]:
all_geophysics.keys()
print(f"Dictionary of all_geophysics includes information about\n\n{list(all_geophysics.keys())}")

In [ ]:
print(f"Within geophysics_data, it contains geophysics lines:\n\n{list(all_geophysics['geophysics_data'].keys())}")


In [ ]:
geophysics_line = "AU-GP163-01"
fig = plot_geophysics(all_geophysics['geophysics_data'][geophysics_line], title=geophysics_line)
fig.show(config={"responsive": False})
print(f"Demo of geophysics data for {geophysics_line}:\n")
all_geophysics['geophysics_data'][geophysics_line]

### geophysics_BH_register

In [ ]:
geophysics_BH_register

### geophysics_bh_results dataframe

In [ ]:
print(f"The following dataframe is merged from geophysics line and its nearby Borehole\n\nThis includes geophysics line of \n\n{geophysics_bh_results_df.Geophysics_ID.unique()}\n")
print("\nDemo of this merged dataframe is: \n")
geophysics_bh_results_df

### geophysics_bh_lab

Dataframe including:
1. Geophysics chainage
2. Geophysics velocity versus depth
3. Borehole consistency versus chainage
4. Borehole geology
5. Lab results

In [ ]:
geophysics_bh_lab

## Results - Visualisation

### Setting geophysics line

In [ ]:
Geophysics_ID = 'AU-GP163-01'
lab_to_plot = 'SPT_N'

### Setting plotting range

In [ ]:
xmin = min(geophysics_data[Geophysics_ID].Chainage.min()-20, geophysics_bh_results_df[geophysics_bh_results_df['Geophysics_ID']==Geophysics_ID].Chainage.min()-20)
xmax = max(geophysics_data[Geophysics_ID].Chainage.max()+20, geophysics_bh_results_df[geophysics_bh_results_df['Geophysics_ID']==Geophysics_ID].Chainage.max()+20)
ymin = min(geophysics_data[Geophysics_ID].To_RL.min()-1, geophysics_bh_results_df[geophysics_bh_results_df['Geophysics_ID']==Geophysics_ID].To_RL.min()-1)
ymax = max(geophysics_data[Geophysics_ID].From_RL.max()+10, geophysics_bh_results_df[geophysics_bh_results_df['Geophysics_ID']==Geophysics_ID].From_RL.max()+10)

### Plot of consistency over geophysics line

In [ ]:
plot_df = geophysics_bh_results_df[geophysics_bh_results_df["Geophysics_ID"]==Geophysics_ID]
Consistency_order = [c for c in CONSISTENCY_ORDER if c in plot_df["Consistency"].unique()]
plot_df.head(3)

In [ ]:
fig = px.scatter(plot_df, x='Chainage', y='From_RL', color='Consistency', height=400,
                 range_x=[xmin,xmax], range_y=[ymin, ymax], 
                 category_orders = {'Consistency':Consistency_order},
                 title=f"Nearby borehole consistency at {Geophysics_ID}",
                 hover_data=HOVER_DATA,
                )
fig.update_xaxes(dtick=25)
fig.update_yaxes(dtick=5)
# fig.update_traces(marker=dict(size=6))
add_label(fig, plot_df)
fig.show(config={"responsive": False})

In [ ]:
fig = px.scatter(plot_df, x='Chainage', y='From_RL', color='Velocity',
                 height=500, range_x=[xmin,xmax], range_y=[ymin, ymax], 
                 color_continuous_scale="jet", range_color=[50,950],
                title=f"Reflection of shearwave velocity (at {Geophysics_ID}) over nearby borehole",
                )
fig.update_layout(coloraxis_colorbar=dict(title="S-Velocity (m/s)", tick0=50, dtick=100))
add_label(fig, plot_df)
fig.update_xaxes(dtick=25)
fig.update_yaxes(dtick=5)
fig.show(config={"responsive": False})

In [ ]:
fig = go.Figure()
add_background_geophysics(all_geophysics, Geophysics_ID, fig, transpareny=0.4)
scatter = px.scatter(plot_df, x='Chainage', y='From_RL', color='Consistency', height=500,
                    # color_discrete_sequence=px.colors.qualitative.Bold,
                    category_orders = {'Consistency':Consistency_order},
                    range_x=[xmin,xmax], range_y=[ymin, ymax], render_mode='webgl',
                    title=f"Overlay of borehole consistency over {Geophysics_ID}",)
fig.add_traces(scatter.data)
add_label(fig, plot_df)
fig.update_xaxes(dtick=25)
fig.update_yaxes(dtick=5)
fig.update_layout(scatter.layout)
fig.show(config={"responsive": False})

### Plot of UCS / SPT over geophysics line

In [ ]:
plot_df = geophysics_bh_lab[geophysics_bh_lab.Geophysics_ID == Geophysics_ID]
plot_df[lab_to_plot] = plot_df[lab_to_plot].astype('str')

In [ ]:
fig = px.scatter(plot_df, x='Chainage', y='From_RL', color=lab_to_plot, height=450, 
                        color_discrete_map={'nan':'rgba(0,0,0,0.9)'},
                        range_x = [xmin, xmax],
                        range_y=[ymin, ymax], title=f"{lab_to_plot} at {Geophysics_ID}",
                        render_mode='webgl')
add_label(fig, plot_df)
fig.update_xaxes(dtick=25)
fig.update_yaxes(dtick=5)
fig.show(config={"responsive": False})

In [ ]:
fig = px.scatter(plot_df, x='Chainage', y='From_RL', color='Velocity', height=450,
                        range_x = [xmin, xmax],
                        range_y=[ymin, ymax], 
                        title=f"Reflection of shearwave velocity (at {Geophysics_ID}) over BH",
                        color_continuous_scale="jet", range_color=[50,950],
                        render_mode='webgl')
add_label(fig, plot_df)
fig.update_xaxes(dtick=25)
fig.update_yaxes(dtick=5)
fig.show(config={"responsive": False})

In [ ]:

fig = go.Figure()
add_background_geophysics(all_geophysics, Geophysics_ID, fig, transpareny=0.4)
scatter = px.scatter(plot_df, x='Chainage', y='From_RL', color=lab_to_plot, height=450,
                        color_discrete_map={'nan':'rgba(0,0,0,0.9)'},
                        range_x = [xmin, xmax],
                        range_y=[ymin, ymax], 
                        title=f"Overlay of {lab_to_plot} over {Geophysics_ID}",
                        render_mode='webgl')
add_label(scatter, plot_df)
fig.update_xaxes(dtick=25)
fig.update_yaxes(dtick=5)
fig.add_traces(scatter.data)
fig.update_layout(scatter.layout)
fig.show(config={"responsive": False})

## Statistical analysis 

In [ ]:
geophysics_bh_results_df.head(3)

### Consistency versus velocity

In [ ]:
Consistency_order = [c for c in CONSISTENCY_ORDER if c in geophysics_bh_results_df["Consistency"].unique()]
px.scatter(geophysics_bh_results_df, y='Consistency', x='Velocity',category_orders= {'Consistency':Consistency_order}, 
           color='soil_type', marginal_x='box')

In [ ]:
fig = px.box(
    geophysics_bh_results_df, 
    y="Consistency", x="Velocity",
    category_orders={"Consistency": Consistency_order},
    points="all",  # show all data points overlaid
    title="Velocity distribution by Consistency",
    height = 600,
    color='soil_type'
)
fig.show()

In [ ]:
import pandas as pd

summary = geophysics_bh_results_df.groupby("Consistency")["Velocity"].agg(["mean","std"]).reset_index()

fig = px.bar(
    summary, y="Consistency", x="mean", 
    error_x="std", 
    category_orders={"Consistency": Consistency_order},
    title="Mean velocity ± std by consistency",labels={"mean":"Mean Velocity (m/s)"},
)
fig.show()

### SPT versus velocity

In [ ]:
geophysics_bh_lab

In [ ]:
SPT = geophysics_bh_lab[geophysics_bh_lab.SPT_N.notna()]
SPT.head(3)

In [ ]:
# X = sm.add_constant(SPT["SPT_N"])
# Y = SPT["Velocity"]
# model = sm.OLS(Y, X).fit()
# a = model.params["SPT_N"]      # slope
# b = model.params["const"]    # intercept
# r2 = model.rsquared          # R²
# print(model.summary())

In [ ]:
# Consistency_order = [c for c in CONSISTENCY_ORDER if c in geophysics_bh_lab["Consistency"].unique()]
fig = px.scatter(SPT, x='SPT_N', y='Velocity', trendline="ols", 
                 facet_col='soil_type',
                 title=f"SPT vs Velocity (m/s)", 
                 )

# fig.add_annotation(
#     x=0.05, y=0.95, xref="paper", yref="paper",
#     text=f"y = {a:.2f}x + {b:.2f}<br>R² = {r2:.2f}",
#     showarrow=False,
#     font=dict(size=12, color="black"),
#     align="left",
#     bordercolor="black", borderwidth=1, bgcolor="white", opacity=0.8
# )

fig.show()

### UCS versus velocity

In [ ]:
UCS = geophysics_bh_lab[geophysics_bh_lab.UCS_MPa.notna()]
UCS.head(3)

In [ ]:
# X = sm.add_constant(UCS["UCS_MPa"])
# Y = UCS["Velocity"]
# model = sm.OLS(Y, X).fit()
# a = model.params["UCS_MPa"]      # slope
# b = model.params["const"]    # intercept
# r2 = model.rsquared          # R²
# print(model.summary())

In [ ]:
Consistency_order = [c for c in CONSISTENCY_ORDER if c in geophysics_bh_lab["Consistency"].unique()]
fig = px.scatter(UCS, x='UCS_MPa', y='Velocity',trendline="ols")

# fig.add_annotation(
#     x=0.05, y=0.95, xref="paper", yref="paper",
#     text=f"y = {a:.2f}x + {b:.2f}<br>R² = {r2:.2f}",
#     showarrow=False,
#     font=dict(size=12, color="black"),
#     align="left",
#     bordercolor="black", borderwidth=1, bgcolor="white", opacity=0.8
# )

fig.show()